# Resource Tracker - Leak Detection for Lionherd Objects

Resource tracker provides lightweight leak detection for long-lived objects. It uses weakref-based automatic cleanup to track resource lifetimes without preventing garbage collection.

**Core Features:**
- **Automatic Cleanup**: Uses `weakref.finalize` for zero-overhead tracking
- **Thread-Safe**: Synchronous tracking with `threading.Lock`
- **Minimal Overhead**: Tracks by object ID, no strong references
- **Lifecycle Visibility**: Query live resources at any time
- **Sync-Only Design**: Intentionally blocking for sync contexts

**Warning**: This tracker uses `threading.Lock` and is designed for SYNC contexts only. Do NOT call from async code as it may block the event loop.

In [1]:
import gc
import time

from lionherd_core.libs.concurrency import (
    LeakTracker,
    track_resource,
)

## 1. Basic Resource Tracking

Track objects with automatic cleanup when they're garbage collected.

In [2]:
# Create a tracker
tracker = LeakTracker()


# Create an object to track
class Connection:
    def __init__(self, name):
        self.name = name


conn = Connection("db-connection")

print(f"Before tracking: {len(tracker.live())} live resources")

# Track it
tracker.track(conn, name="connection-1", kind="database")

print(f"After tracking: {len(tracker.live())} live resources")
print(f"Tracked: {tracker.live()[0]}")

Before tracking: 0 live resources
After tracking: 1 live resources
Tracked: LeakInfo(name='connection-1', kind='database', created_at=1762706468.5488179)


## 2. Automatic Cleanup via Garbage Collection

When the tracked object is garbage collected, it's automatically removed from tracking.

In [3]:
print(f"Before deletion: {len(tracker.live())} live resources")

# Delete the object
del conn

# Force garbage collection
gc.collect()

print(f"After deletion and GC: {len(tracker.live())} live resources")
print("✓ Object automatically untracked on garbage collection")

Before deletion: 1 live resources
After deletion and GC: 0 live resources
✓ Object automatically untracked on garbage collection


## 3. Manual Untracking

You can explicitly untrack objects before they're garbage collected.

In [4]:
# Track a new object
conn2 = Connection("temp-connection")
tracker.track(conn2, name="connection-2", kind="temporary")

print(f"After tracking: {len(tracker.live())} live resources")

# Manually untrack
tracker.untrack(conn2)

print(f"After manual untrack: {len(tracker.live())} live resources")
print(f"Object still exists: {conn2}")

After tracking: 1 live resources
After manual untrack: 0 live resources
Object still exists: <__main__.Connection object at 0x11387a810>


## 4. Tracking Multiple Resources

Track multiple objects simultaneously and query them by type.

In [5]:
# Create multiple resources
resources = [
    (Connection("db1"), "db-conn-1", "database"),
    (Connection("db2"), "db-conn-2", "database"),
    (Connection("cache1"), "cache-conn-1", "cache"),
    (Connection("cache2"), "cache-conn-2", "cache"),
    (Connection("mq1"), "mq-conn-1", "message_queue"),
]

# Track them all
for obj, name, kind in resources:
    tracker.track(obj, name=name, kind=kind)

# Query live resources
live = tracker.live()
print(f"Total live resources: {len(live)}")

# Group by kind
by_kind = {}
for info in live:
    by_kind.setdefault(info.kind, []).append(info)

for kind, infos in by_kind.items():
    print(f"\n{kind.replace('_', ' ').title()} connections: {len(infos)}")
    for info in infos:
        print(f"  - {info.name} ({info.kind})")

Total live resources: 5

Database connections: 2
  - db-conn-1 (database)
  - db-conn-2 (database)

Cache connections: 2
  - cache-conn-1 (cache)
  - cache-conn-2 (cache)

Message Queue connections: 1
  - mq-conn-1 (message_queue)


## 5. Lifecycle Timestamps

Each tracked resource includes a creation timestamp for age analysis.

In [6]:
# Create a long-lived resource
old_conn = Connection("old")
tracker.track(old_conn, name="long-lived", kind="database")

# Wait a bit
time.sleep(1.0)

# Check ages
now = time.time()
live = tracker.live()

print("Resource ages (seconds):")
for info in live:
    age = now - info.created_at
    print(f"  {info.name}: {age:.2f}s")

# Find old resources (potential leaks)
threshold = 1.0  # seconds
old_resources = [info for info in live if (now - info.created_at) > threshold]

print(f"\nOld resources (>{threshold}s): {len(old_resources)}")
for info in old_resources:
    age = now - info.created_at
    print(f"  - {info.name}: {age:.2f}s old")

Resource ages (seconds):
  db-conn-1: 1.01s
  db-conn-2: 1.01s
  cache-conn-1: 1.01s
  cache-conn-2: 1.01s
  mq-conn-1: 1.01s
  long-lived: 1.00s

Old resources (>1.0s): 6
  - db-conn-1: 1.01s old
  - db-conn-2: 1.01s old
  - cache-conn-1: 1.01s old
  - cache-conn-2: 1.01s old
  - mq-conn-1: 1.01s old
  - long-lived: 1.00s old


## 6. Module-Level Convenience Functions

Use `track_resource()` and `untrack_resource()` for a singleton tracker.

In [7]:
# Module-level functions use a global singleton tracker
global_conn = Connection("global")
track_resource(global_conn, name="global-conn", kind="database")

# Note: We can't directly query the global tracker from here,
# but the tracking is active
print("Before: global tracker has resources")

# Clean up
tracker.clear()
print(f"After clear: {len(tracker.live())} live resources")

# Track a few more
conn_a = Connection("a")
conn_b = Connection("b")
track_resource(conn_a, name="conn-a", kind="test")
track_resource(conn_b, name="conn-b", kind="test")

print("After tracking with module functions: tracked resources exist")

Before: global tracker has resources
After clear: 0 live resources
After tracking with module functions: tracked resources exist


## 7. Leak Detection Pattern

Use resource tracking to detect leaks in production code.

In [8]:
# Simulate a leaky operation
leak_tracker = LeakTracker()


def process_requests(count):
    """Simulated operation that leaks connections."""
    connections = []
    for i in range(count):
        conn = Connection(f"req-{i}")
        leak_tracker.track(conn, name=f"conn-{i}", kind="request")
        connections.append(conn)  # Bug: not cleaning up!
    return connections  # Leaked references!


# Run the operation
leaked = process_requests(10)
print(f"Created {len(leaked)} connections")

# Check for leaks
live = leak_tracker.live()
print(f"After processing: {len(live)} live connections (expected 0)")

if live:
    print("\n⚠️  LEAK DETECTED!")
    print("Leaked connections:")
    now = time.time()
    for info in live:
        age = now - info.created_at
        print(f"  - {info.name} (age: {age:.2f}s)")

Created 10 connections
After processing: 10 live connections (expected 0)

⚠️  LEAK DETECTED!
Leaked connections:
  - conn-0 (age: 0.00s)
  - conn-1 (age: 0.00s)
  - conn-2 (age: 0.00s)
  - conn-3 (age: 0.00s)
  - conn-4 (age: 0.00s)
  - conn-5 (age: 0.00s)
  - conn-6 (age: 0.00s)
  - conn-7 (age: 0.00s)
  - conn-8 (age: 0.00s)
  - conn-9 (age: 0.00s)


## 8. Thread-Safety Demonstration

Tracker is thread-safe with internal locking.

In [9]:
import threading

# Concurrent tracking
thread_tracker = LeakTracker()
thread_objects = []  # Keep references


def worker(thread_id, count):
    """Worker that tracks resources concurrently."""
    for i in range(count):
        conn = Connection(f"thread-{thread_id}-{i}")
        thread_tracker.track(conn, name=f"t{thread_id}-conn-{i}", kind="worker")
        thread_objects.append(conn)  # Keep alive


# Run concurrent workers
threads = []
for tid in range(5):
    t = threading.Thread(target=worker, args=(tid, 20))
    threads.append(t)
    t.start()

# Wait for completion
for t in threads:
    t.join()

print(f"Started {len(threads)} threads, each creating 20 connections")
print(f"After concurrent tracking: {len(thread_tracker.live())} live resources")

# Cleanup
thread_objects.clear()
gc.collect()

print(f"After cleanup: {len(thread_tracker.live())} live resources")
print("✓ Thread-safe tracking verified")

Started 5 threads, each creating 20 connections
After concurrent tracking: 100 live resources
After cleanup: 0 live resources
✓ Thread-safe tracking verified


## 9. Clear All Tracked Resources

Bulk clear for testing or reset scenarios.

In [10]:
# Track some resources
clear_tracker = LeakTracker()
clear_objects = []

for i in range(10):
    obj = Connection(f"clear-test-{i}")
    clear_tracker.track(obj, name=f"obj-{i}", kind="test")
    clear_objects.append(obj)

print(f"Before clear: {len(clear_tracker.live())} live resources")

# Clear all
clear_tracker.clear()

print(f"After clear: {len(clear_tracker.live())} live resources")
print("Objects still exist but are no longer tracked")

Before clear: 10 live resources
After clear: 0 live resources
Objects still exist but are no longer tracked


## 10. Production Usage Pattern

Typical production integration with context managers.

In [11]:
class ConnectionPool:
    """Example connection pool with leak tracking."""

    _tracker = LeakTracker()

    def __init__(self, name, size=10):
        self.name = name
        self.size = size
        self.connections = [Connection(f"{name}-{i}") for i in range(size)]
        # Track the pool itself
        self._tracker.track(self, name=name, kind="connection_pool")

    def __enter__(self):
        return self

    def __exit__(self, *args):
        # Manual cleanup
        self.connections.clear()
        self._tracker.untrack(self)

    @classmethod
    def check_leaks(cls):
        """Check for leaked pools."""
        return cls._tracker.live()


# Use with context manager
with ConnectionPool("api-pool", size=5) as pool:
    print(f"During context: {len(ConnectionPool.check_leaks())} live pool")

print(f"After exit: {len(ConnectionPool.check_leaks())} live pools")
print("✓ Automatic cleanup on context exit")

During context: 1 live pool
After exit: 0 live pools
✓ Automatic cleanup on context exit


## Summary Checklist

**Resource Tracker Essentials:**
- ✅ Weakref-based tracking (no strong references)
- ✅ Automatic cleanup via `weakref.finalize`
- ✅ Thread-safe with `threading.Lock`
- ✅ Manual tracking/untracking
- ✅ Lifecycle timestamps for age analysis
- ✅ Group resources by type/kind
- ✅ Detect leaks via live resource queries
- ✅ Module-level singleton for convenience
- ✅ Sync-only design (blocking lock)

**Best Practices:**
- Use for debugging and testing, not hot paths
- Track long-lived resources (connections, pools, sessions)
- Set age thresholds to detect stale resources
- Clear tracker in test teardown
- Never call from async code (blocking lock)

**Next Steps:**
- See `AsyncManager` for async resource management
- See `Event` for tracking state changes
- See `Flow` for workflow lifecycle management